## check recent year values

In [1]:
import os, sys
import geopandas as gpd
import datetime
import matplotlib.pyplot as plt
from pw_data import SIC25k

In [2]:

# Define constants
CRS = 'epsg:3413'  # Polar Stereographic North projection
NRT_DAILY_ID = 'nsidcG10016v3nh1day'  # Dataset ID for near real-time sea ice data
VAR_NAME = 'cdr_seaice_conc'  # Variable name for sea ice concentration
GRID_CELL_AREA_ID = 'pstere_gridcell_N25k'  # Grid cell area ID for sea ice extent calculations

# Define regions and corresponding shapefiles
REGIONS = dict([('AlaskanArctic', 'arctic_sf.shp')])
                # ,
                # ('NorthernBering', 'nbering_sf.shp'),
                # ('EasternBering', 'ebering_sf.shp'),
                # ('SoutheasternBering', 'se_bering_sf.shp')])

In [3]:
# Instantiate SIC25k object and load grid area data
sic_m = SIC25k(NRT_DAILY_ID, VAR_NAME, CRS)


Unable to load data: 'NoneType' object is not subscriptable


In [4]:
sic_m.load_area(GRID_CELL_AREA_ID)  

In [5]:
# If the current month is September (9) or later, use the current year, otherwise use the previous year.
today = datetime.date.today()
lastyear = today.year if today.month > 8 else today.year - 1

In [6]:
lastyear


2024

In [ ]:



# Loop over each region and its corresponding shapefile
for name, shp in REGIONS.items():
    print(f'Processing {name} region...')
    # Load the shapefile and project it to the dataset's CRS
    alaska_shp = gpd.read_file(f'resources/akmarineeco/{shp}')
    alaska_shp_proj = alaska_shp.to_crs(CRS)

    # Subset the dataset by time and region (ex: 2023-09-01 to 2025-08-31)
    ds, area = sic_m.subset_dim([f'{lastyear-1}-09-01', f'{lastyear+1}-12-31'], alaska_shp_proj)

    # Format sea ice concentration data to binary using a 0.15 threshold
    sic = sic_m.format_sic(ds, 0.15)

    # Compute sea ice extent in square kilometers
    ext = sic_m.compute_extent_km(sic, area)
    
    # Clean up memory to avoid excessive usage
    del alaska_shp, alaska_shp_proj, ds, sic 

    # Convert xarray object to a dataframe, reformat, and save as CSV
    ext_df = (ext
                .to_dataframe()
                .reset_index()
                .drop(['spatial_ref'], axis='columns')
                .rename(columns={'time': 'date'}))
 
    ext_df.to_csv(f'nrt_extent_{name}.csv', index=False)  

Processing AlaskanArctic region...
<xarray.DataArray 'seaice_extent' (time: 358, ygrid: 56, xgrid: 33)> Size: 5MB
dask.array<getitem, shape=(358, 56, 33), dtype=float64, chunksize=(1, 56, 33), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 3kB 2024-07-01 2024-07-02 ... 2025-06-23
    spatial_ref  int64 8B 0
  * ygrid        (ygrid) float64 448B 1.462e+06 1.438e+06 ... 1.125e+05 8.75e+04
  * xgrid        (xgrid) float64 264B -2.338e+06 -2.312e+06 ... -1.538e+06
date
2024-07-01    1848
2025-02-10    1848
2025-03-02    1848
2025-03-01    1848
2025-02-28    1848
              ... 
2024-10-26    1848
2024-10-25    1848
2024-10-24    1848
2024-10-23    1848
2025-06-23    1848
Name: count, Length: 358, dtype: int64
